In [ ]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from qdrant_client import QdrantClient
from llama_index.core.retrievers import VectorIndexRetriever
import re

In [ ]:
# ---------- CONNECT TO QDRANT ----------
client = QdrantClient(url="http://localhost:6333")

In [ ]:
# ---------- CONFIG ----------
COLLECTION = "saudi_labor_law"

In [ ]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
# ---------- LOAD EXISTING INDEX ----------
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context,
    embed_model=embed_model
)

In [ ]:
# ---------- CREATE RETRIEVER ----------
retriever = VectorIndexRetriever(index=index, similarity_top_k=5)

In [ ]:
def detect_lang(text):
    return "ar" if re.search(r'[\u0600-\u06FF]', text) else "en"

def answer_query_auto(query: str):
    lang = detect_lang(query)
    results = retriever.retrieve(query)

    print(f"\n🔍 Query ({'Arabic' if lang=='ar' else 'English'}): {query}")
    for rank, r in enumerate(results, start=1):
        md = r.node.metadata

        print(f"\n--- Result {rank} | {md.get('article_name', '')}")
        if lang == "ar":
            print(f"📘 {md.get('arabic_name')} | {md.get('chapter_title_ar')} |  {md.get('part_title_ar')}")
            print(f"AR:  {md.get('arabic_content')}")
            print(f"EN: {md.get('english_content')[:120]}...\n")
        else:
            print(f"📘 {md.get('arabic_name')} | {md.get('chapter_title_ar')} |  {md.get('part_title_ar')}")
            print(f"AR:  {md.get('arabic_content')}")
            print(f"EN: {md.get('english_content')}...\n")

In [ ]:
answer_query_auto("What does Article 23 say?")

In [ ]:
answer_query_auto(" المادة العشرين ")

In [ ]:
answer_query_auto("في 5 سنوات كم يستحق العامل رصيد الإجازات")

In [ ]:
answer_query_auto(" أجر اضافي")